In [11]:
import pandas as pd
import numpy as np
from gensim.test.utils import datapath
from gensim import utils
import gensim.downloader as api
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize


In [ ]:
DATA_PATH = './data.tsv'

In [ ]:
class AmazonCorpus:
    def __iter__(self):
        

In [5]:
google_w2v_model = api.load('word2vec-google-news-300')

In [8]:
result = google_w2v_model.most_similar(positive=['king', 'woman'], negative=['man'], topn=1)
print("King - Man + Woman =", result[0])

similarity = google_w2v_model.similarity('excellent', 'outstanding')
print("Similarity between 'excellent' and 'outstanding':", similarity)

result = google_w2v_model.most_similar(positive=['fastest', 'slow'], negative=['fast'], topn=1)
print("Fastest - Fast + Slow =", result[0])

King - Man + Woman = ('queen', 0.7118193507194519)
Similarity between 'excellent' and 'outstanding': 0.5567486
Fastest - Fast + Slow = ('slowest', 0.7025300860404968)


In [9]:
review_data = pd.read_table('data.tsv', on_bad_lines='skip', names=['star_rating', 'review_body'])

C:\Users\jehil\AppData\Local\Temp\ipykernel_15176\1927948543.py:1: DtypeWarning: Columns (1,4,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  review_data = pd.read_table('data.tsv', on_bad_lines='skip', names=['star_rating', 'review_body'])


In [13]:
review_data['review_body'] = review_data['review_body'].str.lower()
review_data['tokens'] = review_data['review_body'].apply(word_tokenize)

TypeError: expected string or bytes-like object

In [ ]:
embedding_size = 300
window_size = 13
min_word_count = 9

amazon_w2v_model = Word2Vec(sentences=review_data['tokens'], vector_size=embedding_size, window=window_size, min_count=min_word_count)